In [9]:
import numpy as np
import pandas as pd
from collections import Counter
import requests
from bs4 import BeautifulSoup
#import musicalbeeps
import pandas as pd
import json

In [74]:
# Find a list of guitar tabs you find interesting. These are country tabs! Ultimate Guitar Tabs loads 100 pages, or 5000 songs
pages = "https://www.ultimate-guitar.com/explore?genres[]=49&page="
URL = []
for i in range(1, 100):
    url = pages + str(i)
    page = requests.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    
    element = soup.find('div', class_='js-store')
    data_content = element['data-content']
    data_dict = json.loads(data_content)
    songs = data_dict.get('store').get('page').get('data').get('data').get('tabs')
    
    for song in songs:
        URL.append(song.get('tab_url'))


In [75]:
len(URL)

4702

In [77]:
# Web Scraping
import requests
import json
from bs4 import BeautifulSoup
from concurrent.futures import ThreadPoolExecutor, as_completed

# Function to fetch and process each URL
def fetch_and_process(url):
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, 'html.parser')

        element = soup.find('div', class_='js-store')
        
        if element:
            data_content = element['data-content']
            data_dict = json.loads(data_content)
            new_dict = data_dict.get('store')
            
            # Extracting chords
            new_chords = list(new_dict.get('page').get('data').get('tab_view').get('applicature').keys())
            return new_chords
        else:
            print(f"Element not found for URL {url}")
            return []
        
    except Exception as e:
        print(f"URL {url} didn't work: {str(e)}")
        return []

# List to store all chords
all_chords = []

# Using ThreadPoolExecutor to parallelize the URL fetching
with ThreadPoolExecutor(max_workers=20) as executor:  # You can adjust max_workers as needed
    # Submitting all tasks to the executor
    futures = [executor.submit(fetch_and_process, url) for url in URL]

    # Processing results as they complete
    for future in as_completed(futures):
        result = future.result()
        if result:
            all_chords.append(result)


URL https://tabs.ultimate-guitar.com/tab/big-thief/velvet-ring-tabs-2847782 didn't work: 'NoneType' object has no attribute 'keys'
URL https://tabs.ultimate-guitar.com/tab/chris-isaak/wicked-game-tabs-274609 didn't work: 'NoneType' object has no attribute 'keys'
URL https://tabs.ultimate-guitar.com/tab/big-thief/velvet-ring-tabs-3668306 didn't work: 'NoneType' object has no attribute 'keys'
URL https://tabs.ultimate-guitar.com/tab/johnny-cash/hurt-tabs-47818 didn't work: 'NoneType' object has no attribute 'keys'
URL https://tabs.ultimate-guitar.com/tab/chris-stapleton/tennessee-whiskey-tabs-1741208 didn't work: 'NoneType' object has no attribute 'keys'
URL https://tabs.ultimate-guitar.com/tab/colter-wall/the-devil-wears-a-suit-and-tie-tabs-2612178 didn't work: 'NoneType' object has no attribute 'keys'
URL https://tabs.ultimate-guitar.com/tab/johnny-cash/folsom-prison-blues-tabs-332252 didn't work: 'NoneType' object has no attribute 'keys'
URL https://tabs.ultimate-guitar.com/tab/john-d

In [82]:
bigrams_webscraped = []

# Loop through each list of chords
for chord_list in all_chords:
    # Get 2-chord sequences from each list
    for i in range(len(chord_list) - 1):
        bigrams_webscraped.append(f"{chord_list[i]} {chord_list[i + 1]}")


In [85]:
chords_flattened = [item for sublist in all_chords for item in sublist]

In [92]:
all_chords_no_repeats = list(set(chords_flattened))

In [103]:
def predict_next_state(chord, bigrams):
    bigrams_with_current_chord = [bigram for bigram in bigrams if bigram.split(' ')[0]==chord]

    count_appearance = dict(Counter(bigrams_with_current_chord))
    
    for ngram in count_appearance.keys():
        count_appearance[ngram] = count_appearance[ngram]/len(bigrams_with_current_chord)

    options = [key.split(' ')[1] for key in count_appearance.keys()]

    probabilities = list(count_appearance.values())

    return np.random.choice(options, p=probabilities)

In [125]:
def generate_sequence(chord, bigrams, length=4):
    chords = []
    for n in range(length):
        chords.append(predict_next_state(chord, bigrams))
        chord = chords[len(chords)-1]
    return chords

In [126]:
def make_markov_chain(chords_no_repeats, bigrams):    
    markov_chain = {}

    for i in range(len(chords_no_repeats)):
        markov_chain.update({chords_no_repeats[i]:{}})
        for j in range(len(chords_no_repeats)):
            markov_chain.get(chords_no_repeats[i]).update({chords_no_repeats[j]:0})



    for i in range(len(chords_no_repeats)):

        bigrams_with_current_chord = [bigram for bigram in bigrams if bigram.split(' ')[0]==chords_no_repeats[i]]
        count_appearance = dict(Counter(bigrams_with_current_chord))
        for ngram in count_appearance.keys():
            count_appearance[ngram] = count_appearance[ngram]/len(bigrams_with_current_chord)
        options = [key.split(' ')[1] for key in count_appearance.keys()]
        probabilities = list(count_appearance.values())

        for j in range(len(options)):
            markov_chain.get(chords_no_repeats[i])[options[j]] = probabilities[j]


In [127]:
data = make_markov_chain(all_chords_no_repeats, bigrams_webscraped) 

In [128]:
sequence = generate_sequence('Am', bigrams_webscraped)

In [129]:
sequence

['A7', 'D/F#', 'D', 'D7']

In [147]:
from midiutil import MIDIFile
import pygame
import time
import music21 as m21

midi = MIDIFile(1)
track = 0
time = 0
midi.addTrackName(track, time, "Chord Progression")
midi.addTempo(track, time, 120)  # Tempo in BPM

channel = 0
volume = 100
duration = 1  # Duration of each chord

# List of chords to play
chords = generate_sequence('C', bigrams_webscraped, 50)
chords = [chord.replace('b', '-') for chord in chords]

print(chords)
# Function to convert chord name to MIDI notes using music21
def chord_to_midi_notes(chord_name):
    chord = m21.harmony.ChordSymbol(chord_name)
    midi_notes = [p.midi for p in chord.pitches]
    return midi_notes

for i in range(5):
    for chord in chords:
        midi_notes = chord_to_midi_notes(chord)
        for note in midi_notes:
            midi.addNote(track, channel, note, time, duration, volume)
        time += duration  # Move to the next chord

with open("chord_progression.mid", "wb") as output_file:
    midi.writeFile(output_file)

pygame.mixer.init()
pygame.mixer.music.load("chord_progression.mid")
pygame.mixer.music.play()



['D', 'G', 'A', 'G', 'C', 'G', 'C/G', 'D7/F#', 'D7', 'Am', 'Am7', 'D', 'G', 'D/F#', 'Em', 'Am', 'D', 'G', 'D', 'G', 'Am', 'C', 'G', 'D', 'Cadd9', 'B-sus2', 'A7', 'D', 'Em', 'D', 'A', 'B', 'C#', 'C', 'Dm', 'D7', 'D', 'G', 'D', 'A', 'Bm', 'A', 'G', 'D', 'D7', 'G7', 'G', 'A7', 'D/F#', 'Em']
